In [7]:
import numpy as np
import os
current_dir= os.getcwd()
data_dir= current_dir+'/npz_data/processed_data.npz'
data = np.load(data_dir)
processed_data = data['data']
T, N, M = processed_data.shape
returns = processed_data[:, :, 0]
chara_data= processed_data[:,:,1:]
rf_dir= current_dir+'/npz_data/riskfreerate.npz'
fama_french_data= np.load(rf_dir)
fama_french_data= fama_french_data['fama_french_data']
r_f= fama_french_data[:,4]
r_f= r_f/100
date= fama_french_data[:,0]
date_start= np.asscalar(np.argwhere(date==198402))
date_end= np.asscalar(np.argwhere(date==201401))
returns= returns- (r_f[date_start:date_end:,np.newaxis])
D= 10
# The dimension of returns: T*N
# chara_data = processed_data[:, :, [8, 17, 20, 33, 37]]
# chara_data = processed_data[:, :, [8, 17, 33, 37]]
# chara_data = processed_data[:, :, [8, 17, 20, 33]]
# chara_data = processed_data[:, :, 5:]
L= chara_data.shape[2]

In [9]:
portfolio= np.zeros((T-1, L*D))
for i in range(T-1):
    for j in range(L):
        chara_data_t= chara_data[i,:,j]
        index= np.argsort(chara_data_t)
        # print (index)
        length= int(len(index)/D)
        # print (length)
        for k in range(D):
            index_k= index[int(length*k):int(length*(k+1))]
            portfolio[i, j*D+k]= np.mean(returns[i,index_k])

In [20]:
from __future__ import print_function
def compute_sharpe(K, portfolio, RP= True, Corr= True, Time= True, delta= 600, Shrink= True):
    import scipy.linalg as la
    T, M = portfolio.shape
    window= 120
    optimal_return = np.zeros((T-window))
    gamma = 10    
    ones_T= np.ones((window, 1))
    for t in range(T-window):
        portfolio_t = portfolio[t:(t + window), :]
        if Time: 
            if RP:
                covmat_t= (np.identity(window)+ones_T.dot(ones_T.T)/(window*(2+window))).dot(portfolio_t)
                covmat_t= (covmat_t.dot(portfolio_t.T)).dot(np.identity(window)+ones_T.dot(ones_T.T)/(window*(2+window)))       
            else: 
                covmat_t= np.cov(portfolio_t)        
        else:
            if RP:
                covmat_t = portfolio_t.T.dot(portfolio_t) 
                portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
                covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t)/window * gamma
            else:
                covmat_t= np.cov(portfolio_t.T)
        if Corr:
            covmat_t_diag= np.diag(1./np.sqrt(np.diag(covmat_t)))
            covmat_t= covmat_t_diag.dot(covmat_t.dot(covmat_t_diag))
        if Time:
            variance_t, Factor_t= la.eigh(covmat_t, eigvals=(window-K, window-1))
            Factor_t= Factor_t[:,:K]
            loading_t= Factor_t.T.dot(portfolio_t)
            loading_t= loading_t.T
        else:
            variance_t, loading_t= la.eigh(covmat_t, eigvals=(M-K, M-1))
            Factor_t= portfolio_t.dot(loading_t)
        if Shrink:
            for i in range(K):
                threshold= np.partition(abs(loading_t[:,i]),-delta)[-delta]
                loading_t[np.abs(loading_t[:,i])<threshold,i]=0
                loading_t[:,i]= loading_t[:,i]/np.linalg.norm(loading_t[:,i])
            Factor_t= portfolio_t.dot(loading_t).dot(np.linalg.inv(loading_t.T.dot(loading_t)))
                
        mu= np.mean(Factor_t, axis= 0)
        covmat_factor= np.cov(Factor_t.T)
        if K>=2:
            weight= np.linalg.inv(covmat_factor).dot(mu)
        else:
            weight= mu/covmat_factor
        if Time:        
            optimal_return[t] = (portfolio[t + window, :].dot(loading_t)).dot(np.linalg.inv(loading_t.T.dot(loading_t))).dot(weight)
        else:
            optimal_return[t] = (portfolio[t + window, :].dot(loading_t)).dot(np.linalg.inv(loading_t.T.dot(loading_t))).dot(weight)
    return optimal_return
def easy_sharpe(optimal_return):
        mean_optimal= np.mean(optimal_return)
        cov_optimal= np.cov(optimal_return)
        return abs(mean_optimal)/np.sqrt(cov_optimal)
K= 5
optimal_sharpe= np.zeros((K))
for i in range(K):
#     print (args.Shrink)
    optimal_sharpe[i]= easy_sharpe(compute_sharpe(i+1, portfolio, 1, 1, 1, 0, 0))
print (optimal_sharpe)
# optimal_sharpe

[ 0.15597989  0.15046161  0.27774963  0.37278094  0.36693536]


In [22]:
gamma= 10
# portfolio_demean = portfolio - np.mean(portfolio, axis=0)
# Dimension of portfolio: T*M
covmat = portfolio.T.dot(portfolio) 
portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
variance, loading = np.linalg.eig(covmat)
output= []
for K in range(1, 20):
    loading_chosen = loading[:, :K]
    Factor = portfolio.dot(loading_chosen)
    mu = np.mean(Factor, axis=0)
    covmat_factor = np.cov(Factor.T)
    if K==1:
        output.append (abs(mu)*covmat_factor**(-1/2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print (''.join(twodecimals))

&0.35&0.21&0.46&0.47&0.55&0.59&0.59&0.60&0.62&0.62&0.63&0.64&0.64&0.64&0.64&0.64&0.68&0.68&0.68
